# 1. FAO Penman-Montieth method for estimating Ref. Evapotranspiration

In [1]:
import pandas as pd
import datetime
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('Irrigation_Scheduling_Field70_2023.xlsx', 'Penman', skiprows=2)
df=df.iloc[:, : 6] 
df['Date'] = pd.to_datetime(df['Date'])
df["month1"]=df["Date"].dt.month
#df

In [3]:
# monthly_averages = df.groupby(["month1"]).aggregate({"Air Temperature":np.mean})
#df2 = monthly_averages.rename(columns={'Air Temperature':"Temp_avg"})
#df2

In [4]:
data = {'month1': np.arange(1,13),
        'Temp_avg': [-3.150,-1.345,4.547,11.109,17.016,21.745,22.969,21.738,18.330,11.936,5.572,-0.620]}
 
# Create DataFrame
df2 = pd.DataFrame(data)

# Create a new column and calculate its values using if-else conditions
df2['heat_Flux'] = 0.07 * np.where(df2['month1'] == 1, 
                                    df2['Temp_avg'].shift(-1) - df2['Temp_avg'].iloc[-1],
                                    np.where(df2['month1'] == 12,
                                             df2['Temp_avg'].iloc[0] - df2['Temp_avg'].shift(1),
                                             df2['Temp_avg'].shift(-1) - df2['Temp_avg'].shift(1)))
df=pd.merge(df, df2, left_on='month1', right_on='month1')
df

,Date,Wind Speed,Precipitation,Solar Radiation,Air Temperature,Vapor Pressure,month1,Temp_avg,heat_Flux
0,2023-05-01,5.999999,5.842,55,5.9,7.983117,5,17.016,0.74452
1,2023-05-02,0.000000,0.000,119,8.2,7.652452,5,17.016,0.74452
2,2023-05-03,3.500000,0.000,283,8.9,7.600050,5,17.016,0.74452
3,2023-05-04,0.000000,0.000,436,18.2,11.571283,5,17.016,0.74452
4,2023-05-05,0.000000,0.000,439,22.2,11.007096,5,17.016,0.74452
...,...,...,...,...,...,...,...,...,...
125,2023-09-03,2.600000,0.000,248,25.2,24.425491,9,18.330,-0.68614
126,2023-09-04,2.300000,0.000,225,25.6,24.586407,9,18.330,-0.68614
127,2023-09-05,3.000000,0.000,231,26.0,23.510684,9,18.330,-0.68614
128,2023-09-06,3.899999,1.524,189,23.0,21.858862,9,18.330,-0.68614


In [5]:
df["year"]=df["Date"].dt.year
df['month'] = 1
df['day'] = 1
df['J_Date']=pd.to_datetime(df[["year", "month", "day"]])
df['j']=df['Date'].apply(pd.Timestamp.to_julian_date)-df['J_Date'].apply(pd.Timestamp.to_julian_date)+1


In [6]:
df = df.rename(columns={'Solar Radiation':'radiation','Air Temperature':"Temp",'Wind Speed':'Wind_speed','Vapor Pressure':'V_Pressure'})

#df=df.iloc[0:10,:]


In [7]:
def PET_Penman (Wind_speed, radiation, Temp, V_Pressure, heat_Flux, j):
    lammbda= 2.501-Temp*0.002361 
     
    if Temp > 0:
        Sat_VP = 0.6108 * math.exp(17.27 * Temp / (237.3 + Temp))
    else:
        Sat_VP = V_Pressure * 0.1
        
    vpd= Sat_VP-V_Pressure*0.1
    slope= 4098*0.6108*math.exp(((17.27*Temp)/(Temp+237.3)))/((Temp+237.3)**2)
    
    Pressure= 101.3*((293-0.0065*213.36)/293)**5.26
    gamma  =0.001013*Pressure/(0.622*lammbda)
    sigma=4.903*10**-9
    
    Rbo= sigma*(Temp+273)**4
    Rs= radiation*60*60*24/10e5    
    dr=1+0.033*math.cos(2*math.pi*j/365)
    declination=0.409*math.sin((2*math.pi*j/365)-1.39)
    sunset_angle=math.acos(-1*math.tan(40.469794*math.pi/180)*math.tan(declination))
    Ra=(24*60*0.082/math.pi)*dr*(math.sin(40.469794*math.pi/180)*math.sin(declination)+math.cos(40.469794*math.pi/180)*math.cos(declination)*math.sin(sunset_angle))
    Rso=(213.36*0.00002+0.75)*Ra    
    Rnl=Rbo*(0.34-0.14*math.sqrt(0.1*V_Pressure))*(1.35*(Rs/Rso) - 0.35)      
    Rnet=Rs*(1-0.23)-Rnl
    ETo=(0.408*slope*(Rnet-heat_Flux)+gamma*(900/(Temp+273))*Wind_speed*vpd)/(slope+gamma*(1+0.34*Wind_speed))  
    ETo_in=ETo/25.4
    #return (lammbda,Sat_VP,vpd,slope,Pressure,gamma,sigma,Rbo,Rs,j,dr,declination,sunset_angle,Ra, Rso,Rnl,ETo)
    return ETo, ETo_in


In [8]:
#df['ETo'] = df.apply(lambda row: PET(row['Wind_speed'], row['heat_Flux'], row['radiation'], row['Temp'], row['V_Pressure'],row['j']), axis=1)

# zip allows for multiple columns
%timeit df['ETo_mmDay'], df['ETo_in'] = zip(*df.apply(lambda row: PET_Penman(row['Wind_speed'], row['radiation'], row['Temp'], row['V_Pressure'], row['heat_Flux'],row['j']), axis=1))

#df.ETo_in=df.ETo_in.round(2)
df

3.37 ms ± 50.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


,Date,Wind_speed,Precipitation,radiation,Temp,V_Pressure,month1,Temp_avg,heat_Flux,year,month,day,J_Date,j,ETo_mmDay,ETo_in
0,2023-05-01,5.999999,5.842,55,5.9,7.983117,5,17.016,0.74452,2023,1,1,2023-01-01,121.0,0.976079,0.038428
1,2023-05-02,0.000000,0.000,119,8.2,7.652452,5,17.016,0.74452,2023,1,1,2023-01-01,122.0,1.250607,0.049237
2,2023-05-03,3.500000,0.000,283,8.9,7.600050,5,17.016,0.74452,2023,1,1,2023-01-01,123.0,2.897173,0.114062
3,2023-05-04,0.000000,0.000,436,18.2,11.571283,5,17.016,0.74452,2023,1,1,2023-01-01,124.0,4.588838,0.180663
4,2023-05-05,0.000000,0.000,439,22.2,11.007096,5,17.016,0.74452,2023,1,1,2023-01-01,125.0,4.678842,0.184206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,2023-09-03,2.600000,0.000,248,25.2,24.425491,9,18.330,-0.68614,2023,1,1,2023-01-01,246.0,4.454810,0.175386
126,2023-09-04,2.300000,0.000,225,25.6,24.586407,9,18.330,-0.68614,2023,1,1,2023-01-01,247.0,4.248143,0.167250
127,2023-09-05,3.000000,0.000,231,26.0,23.510684,9,18.330,-0.68614,2023,1,1,2023-01-01,248.0,4.747040,0.186891
128,2023-09-06,3.899999,1.524,189,23.0,21.858862,9,18.330,-0.68614,2023,1,1,2023-01-01,249.0,3.694065,0.145436


## The Ref ET estimated using Penman-Monteith:

In [9]:
# #df.loc[(df.Date >= '2023-08-30') & (df.Date <= '2023-09-10')]
# df=df[['Date','Wind_speed','Precipitation','radiation','Temp','V_Pressure','ETo_mmDay','ETo_in']]
# df